In [1]:
import numpy as np, pandas as pd
from tqdm import tqdm_notebook, tqdm
tqdm.pandas(desc="my bar!")

In [2]:
def extr_data_txt(path):
    with open(path, "r") as f:
        data = []
        for itr, line in tqdm_notebook(enumerate(f)):
            data.append(line.split())
    return data[2:]

In [3]:
train_test_valid_anot = pd.DataFrame(extr_data_txt('data/Eval/list_eval_partition.txt'), 
                                     columns=['Path', 'type'])
train_test_valid_anot.to_csv('data/Anno/train_test_valid_anot.csv' ,index=False)

In [4]:
categories_img = pd.DataFrame(extr_data_txt('data/Anno/list_category_img.txt'), 
                          columns=['Path', 'cat'])
categories_img.to_csv('data/Anno/categories_img.csv' ,index=False)

In [5]:
box_img = pd.DataFrame(extr_data_txt('data/Anno/list_bbox.txt'), 
                          columns=['Path', 'x1', 'y1', 'x2', 'y2'])
box_img.to_csv('data/Anno/bbox_img.csv' ,index=False)

In [6]:
bbox_img = pd.read_csv('data/Anno/bbox_img.csv')

In [7]:
import cv2

In [8]:
def parse_labels(path, x1, y1, x2, y2):
    """
    Definition: Parses label files to extract label and bounding box
        coordinates.  Converts (x1, y1, x1, y2) KITTI format to
        (x, y, width, height) normalized YOLO format.
    Parameters:
    Return: all_labels - contains a list of labels for objects in image
            all_coords - contains a list of coordinate for objects in image
    """
    coords = [x1, y1, x2, y2]
    img_width, img_height, _ = get_img_shape(path)
    if img_width == None:
        return np.nan, np.nan, np.nan, np.nan
    x = float((float(coords[2]) + float(coords[0])) / 2.0) / float(img_width)
    y = float((float(coords[3]) + float(coords[1])) / 2.0) / float(img_height)
    width = float(float(coords[2]) - float(coords[0])) / float(img_width)
    height = float(float(coords[3]) - float(coords[1])) / float(img_height)
    return x, y, width, height

def get_img_shape(path):
    path = 'data/'+path
    img = cv2.imread(path)
    try:
        return img.shape
    except AttributeError:
        return (None, None, None)

In [9]:
bbox_img['x'], bbox_img['y'], bbox_img['width'], bbox_img['height'] = zip(*bbox_img.progress_apply(lambda row: parse_labels(row['Path'], row['x1'], row['y1'],
                                                                                                                            row['x2'], row['y2']), axis=1))

my bar!: 100%|██████████| 289222/289222 [02:18<00:00, 2091.00it/s]


In [10]:
df = bbox_img.merge(train_test_valid_anot).merge(categories_img)

In [11]:
df.to_csv('data/Anno/annotation_w-o_atr.csv', index=False)

In [ ]:
mem = pd.read_csv('data/Anno/Atr_img.csv')